In [1]:
from statsmodels.stats.proportion import proportion_confint
import pandas as pd

from bokeh.io import output_notebook
from bokeh.plotting import figure, show

output_notebook()

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option("display.max_rows", 1000)

Loading BokehJS ...

In [2]:
df = pd.read_csv("recent-grads.csv")

In [3]:
lower, upper = proportion_confint(count=df.ShareWomen * df.Sample_size, nobs=df.Sample_size,
                                  alpha=0.01, method="jeffrey")
ci = pd.DataFrame.from_items([("lower", lower), 
                              ("point", df.ShareWomen),
                              ("upper", upper),
                              ("sample_size", df.Sample_size)]) \
       .set_index(pd.MultiIndex.from_tuples(list(zip(df.Major_category, df.Major)),
                                            names=["category", "major"])) \
       .sort_values("point").sort_index(level="category", sort_remaining=False)
ci

lower  \
category                            major                                                        
Agriculture & Natural Resources     FOOD SCIENCE                                       0.08514   
                                    GENERAL AGRICULTURE                                0.41389   
                                    NATURAL RESOURCES MANAGEMENT                       0.46034   
                                    AGRICULTURAL ECONOMICS                             0.39736   
                                    AGRICULTURE PRODUCTION AND MANAGEMENT              0.51655   
                                    PLANT SCIENCE AND AGRONOMY                         0.48441   
                                    FORESTRY                                           0.50660   
                                    MISCELLANEOUS AGRICULTURE                          0.45997   
                                    SOIL SCIENCE                                       0.18548   
                                    ANIMAL SCIENCES                                    0.85740   
Arts                                COMMERCIAL ART AND GRAPHIC DESIGN                  0.33872   
                                    MISCELLANEOUS FINE ARTS                            0.20562   
                                    MUSIC                                              0.38296   
                                    STUDIO ARTS                                        0.48947   
                                    DRAMA AND THEATER ARTS                             0.56229   
                                    FINE ARTS                                          0.61727   
                                    FILM VIDEO AND PHOTOGRAPHIC ARTS                   0.61902   
                                    VISUAL AND PERFORMING ARTS                         0.58875   
Biology & Life Science              MOLECULAR BIOLOGY                                  0.02600   
                                    NEUROSCIENCE                                       0.30661   
                                    BIOCHEMICAL SCIENCES                               0.41846   
                                    PHARMACOLOGY                                       0.04720   
                                    BOTANY                                             0.16376   
                                    MISCELLANEOUS BIOLOGY                              0.40568   
                                    ENVIRONMENTAL SCIENCE                              0.49887   
                                    BIOLOGY                                            0.56749   
                                    MICROBIOLOGY                                       0.45270   
                                    ZOOLOGY                                            0.45025   
                                    GENETICS                                           0.27435   
                                    ECOLOGY                                            0.51391   
                                    COGNITIVE SCIENCE AND BIOPSYCHOLOGY                0.61879   
                                    PHYSIOLOGY                                         0.81246   
Business                            MISCELLANEOUS BUSINESS & MEDICAL ADMINISTRATION    0.14021   
                                    BUSINESS ECONOMICS                                 0.17675   
                                    ACCOUNTING                                         0.22938   
                                    MANAGEMENT INFORMATION SYSTEMS AND STATISTICS      0.21368   
                                    INTERNATIONAL BUSINESS                             0.21535   
                                    OPERATIONS LOGISTICS AND E-COMMERCE                0.23226   
                                    FINANCE                                            0.32945   
                                    MARKETING AND MARKETING RESEARCH   

In [11]:
plot_df = pd.DataFrame(ci) \
            .assign(name=lambda df: [name if len(name) < 30 else name[:27] + "..."
                                     for name in df.index.get_level_values(1)]) \
            .set_index("name") \
            .sort_values("sample_size").ix[-30:] \
            .sort_values("point")
intervals = [([row.lower, row.upper], [name, name])
             for name, row in plot_df.iterrows()]
xs, ys = tuple(zip(*intervals))
    
p = figure(y_range=list(plot_df.index), title="Gender Ratio for 30 Most Common Majors")
p.xaxis.axis_label = "Gender Ratio (% Female)"
p.multi_line(xs, ys, line_width=5, alpha=0.8)
p.circle(plot_df.point, plot_df.index, size=6, line_color="black", fill_color="white")
show(p)

In [13]:
select = [
    "ACCOUNTING", "ANTHROPOLOGY AND ARCHEOLOGY", "ARCHITECTURE", "ART HISTORY AND CRITICISM",
    "BIOCHEMICAL SCIENCES", "BIOLOGICAL ENGINEERING", "BIOLOGY", "BUSINESS MANAGEMENT AND ADMINISTRATION", 
    "CHEMICAL ENGINEERING", "CHEMISTRY", "CIVIL ENGINEERING", "COMPUTER ENGINEERING", "COMPUTER SCIENCE", 
    "DRAMA AND THEATER ARTS", 
    "ECONOMICS", "ELECTRICAL ENGINEERING", "ENGLISH LANGUAGE AND LITERATURE", 
    "FILM VIDEO AND PHOTOGRAPHIC ARTS", "FINANCE", 
    "GENERAL BUSINESS", 
    "HISTORY", 
    "JOURNALISM", 
    "MATHEMATICS", "MECHANICAL ENGINEERING", "MUSIC", 
    "NURSING", 
    "PHILOSOPHY AND RELIGIOUS STUDIES", "PHYSICS", "POLITICAL SCIENCE AND GOVERNMENT", "PSYCHOLOGY", 
    "SOCIOLOGY", 
]

In [14]:
plot_df = pd.DataFrame(ci) \
            .assign(name=ci.index.get_level_values(1)).set_index("name") \
            .loc[select] \
            .sort_values("point")
intervals = [([row.lower, row.upper], [name, name])
             for name, row in plot_df.iterrows()]
xs, ys = tuple(zip(*intervals))
    
p = figure(y_range=list(plot_df.index), title="Gender Ratio for Selected Majors")
p.xaxis.axis_label = "Gender Ratio (% Female)"
p.multi_line(xs, ys, line_width=5, alpha=0.8)
p.circle(plot_df.point, plot_df.index, size=6, line_color="black", fill_color="white")

show(p)